In [1]:
import json
import os
import sys
import math
import datetime
import pathlib

dir_path = pathlib.Path(os.path.abspath('')).parent.resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

/mnt/OpenFormat/python


In [2]:
file_base = 'core_r1000000_c20'
base_name = file_base
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
os.system(f'python3 gen_workloads.py core 1000000 20 {file_base}')
os.system(f'cp -r {file_base} {PROJ_SRC_DIR}/python/compression')
os.system(f'cp {file_base}/gen_data/{file_base}.csv {PROJ_SRC_DIR}/python/data_gen')

Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!


0

In [4]:
os.chdir(f'{PROJ_SRC_DIR}/python')

In [6]:
workload_lists = ['core']
# workload_lists = ['core', 'geo', 'classic', 'log', 'ml']
base_name = file_base
for workload in workload_lists:
    os.system(f'python3 scripts/parquet_exp.py {base_name} pq_compression -r=false -o=false')

loading config from ./experiments/pq_compression.json ...
----Running experiment: 1 out of 2----
NONE_True_1048576_1048576_2.6_2.0
write parquet time (s): 0.6330640316009521
----Running experiment: 2 out of 2----
ZSTD_True_1048576_1048576_2.6_2.0
write parquet time (s): 0.6999180316925049


In [7]:
# put files into different devices
# devices_list = ['gp2', 'sc1', 'st1','th125', 'th250', 'th500', 'th750', 'th1000', 'nvme']
# devices_list = ['th125', 'th250', 'th500', 'th750', 'th1000', 'nvme']
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_compression.json')
devices_list = ['st1', 'io1', 'gp2', 'th125', 'nvme']
for c in pq_config:
    pq_name_suffix = '_' + "_".join([str(i) for i in list(c.values())]) + '.parquet'
    for d in devices_list:
        os.system(f'cp {base_name}{pq_name_suffix} /{d}/')

loading config from /mnt/OpenFormat/python/experiments/pq_compression.json ...


In [9]:
# running experiments
scan_exec_pq = f'{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/parquet-scan-columnbatch'
os.system('rm outputs/stats.json')
for i in range(5):
    for d in devices_list:
        for workload in workload_lists:
            for conf in pq_config:
                pq_name_suffix = '_' + "_".join([str(i) for i in list(conf.values())]) + '.parquet'
                os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                time.sleep(1)
                # pq_read = float(os.popen(f'''{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/raw_reader_async \
                #         /{d}/{workload}_alltypes{pq_name_suffix} 1''').read().split(':')[-1].strip())
                pq_read, orc_read = [0,0], [0, 0]
                for i in range(2):
                    pq_read[i] = float(os.popen(f'''{scan_exec_pq} \
                        --batch_size=1024 /{d}/{base_name}{pq_name_suffix}''').read().split('\n')[0].split(' ')[-2])
                pq_size = os.path.getsize(f'/{d}/{base_name}{pq_name_suffix}')
                output_stats = conf.copy()
                output_stats['workload'] = workload
                output_stats['time'] = pq_read[0]
                output_stats['time_preload'] = pq_read[1]
                output_stats['size'] = pq_size
                output_stats['device'] = d
                output_stats['i'] = i
                parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv outputs/{}_{}.csv'.format('compression_exp', timestamp))

rm: cannot remove 'outputs/stats.json': No such file or directory


0

In [8]:
# s3 exp
# NOTE: you need to properly set the AWS credentials in your environment variables to run the executable.
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_compression.json')
os.system('rm outputs/stats.json')
for c in pq_config:
    pq_name_suffix = '_' + "_".join([str(i) for i in list(c.values())]) + '.parquet'
    read_time = []
    for i in range(5):
        pq_read = float(os.popen(f'''{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/parquet-scan-s3 \
                        --batch_size=1024 xinyu-test/{base_name}{pq_name_suffix}''').read().split(' ')[-2])
        time.sleep(5)
    read_time.append(pq_read)
    # print average
    print('pq_read: ', sum(read_time)/len(read_time))
    print(pq_name_suffix)

loading config from /mnt/OpenFormat/python/experiments/pq_compression.json ...


rm: cannot remove 'outputs/stats.json': No such file or directory


pq_read:  0.967631
_NONE_True_1048576_1048576_2.6_2.0.parquet
pq_read:  0.899004
_ZSTD_True_1048576_1048576_2.6_2.0.parquet


# Begin param sweep exps

In [8]:
os.chdir(f'{PROJ_SRC_DIR}/python')
car_list = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
def width_list(t):
    if t == 'string':
        width_list = [
                    5,
                    10,
                    25,
                    50,
                    100,
                    250,
                    500,
                    1000
                ]
    elif t == 'int':
        width_list = [
            10,
            100,
            1000,
            10000,
            100000,
            1000000,
            10000000,
            100000000
        ]
    elif t == 'float':
        width_list = [
            100,
            10000,
            1000000,
            100000000
        ]
    return width_list
zipf_list = [
                0.01,
                0.2,
                0.4,
                0.6,
                0.8,
                1.0,
                1.2,
                1.4,
                1.6,
                1.8,
                2.0
            ]
sort_list = [
        0,
                0.1,
                0.2,
                0.3,
                0.4,
                0.5,
                0.6,
                0.7,
                0.8,
                0.9,
                1
            ]
type_list = ['string', 'int', 'float']
fea_list = ['zipf', 'width', 'sort','car']

In [9]:
# generate files
for fea in fea_list:
    for t in type_list:
        if fea == 'width':
            param_list = width_list(t)
        elif fea == 'zipf':
            param_list = zipf_list
        elif fea == 'sort':
            param_list = sort_list
        elif fea == 'car':
            param_list = car_list
        else:
            assert False
        for param in param_list:
            os.system(f'python3 scripts/parquet_exp.py {t}_{fea}{param} pq_compression -r=false -o=false')


loading config from ./experiments/pq_compression.json ...
----Running experiment: 1 out of 2----
NONE_True_1048576_1048576_2.6_2.0
write parquet time (s): 2.4386329650878906
----Running experiment: 2 out of 2----
ZSTD_True_1048576_1048576_2.6_2.0
write parquet time (s): 4.10977578163147
loading config from ./experiments/pq_compression.json ...
----Running experiment: 1 out of 2----
NONE_True_1048576_1048576_2.6_2.0
write parquet time (s): 2.4193053245544434
----Running experiment: 2 out of 2----
ZSTD_True_1048576_1048576_2.6_2.0
write parquet time (s): 4.107470273971558
loading config from ./experiments/pq_compression.json ...
----Running experiment: 1 out of 2----
NONE_True_1048576_1048576_2.6_2.0
write parquet time (s): 2.4076788425445557
----Running experiment: 2 out of 2----
ZSTD_True_1048576_1048576_2.6_2.0
write parquet time (s): 4.211383581161499
loading config from ./experiments/pq_compression.json ...
----Running experiment: 1 out of 2----
NONE_True_1048576_1048576_2.6_2.0
wri

In [12]:
# running experiments
os.system('rm outputs/stats.json')
for i in range(3):
    for fea in fea_list:
        for t in type_list:
            if fea == 'width':
                param_list = width_list(t)
            elif fea == 'zipf':
                param_list = zipf_list
            elif fea == 'sort':
                param_list = sort_list
            elif fea == 'car':
                param_list = car_list
            else:
                assert False
            for param in param_list:
                for conf in pq_config:
                    pq_name_suffix = '_' + "_".join([str(i) for i in list(conf.values())]) + '.parquet'
                    os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
                    time.sleep(1)
                    # pq_read = float(os.popen(f'''{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/raw_reader_async \
                    #         ./{t}_{fea}{param}{pq_name_suffix} 1''').read().split(':')[-1].strip())
                    pq_read, orc_read = [0,0], [0, 0]
                    for i in range(2):
                        pq_read[i] = float(os.popen(f'''{HOME_DIR}/arrow-private/cpp/out/build/openformat-release/release/parquet-scan \
                        --batch_size=1024 {t}_{fea}{param}{pq_name_suffix}''').read().split(' ')[-2])
                    pq_size = os.path.getsize(f'{t}_{fea}{param}{pq_name_suffix}')
                    output_stats = conf.copy()
                    output_stats['feature'] = fea
                    output_stats['type'] = t
                    output_stats['param'] = param
                    output_stats['i'] = i
                    output_stats['time'] = pq_read[0]
                    output_stats['time_preload'] = pq_read[1]
                    output_stats['size'] = pq_size
                    parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv outputs/{}_{}.csv'.format('compression_param_sweep', timestamp))


0